In [61]:
import glob
import pandas as pd
import numpy as np

import os
import re

In [87]:
attack = "crownibp"

In [88]:
results_folder = rf"results\{attack}"

In [89]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]
exclude_datasets = ["Dialysis","divorce","Pbc3","vlbw"]

In [116]:
# CI , IBS , NegLL
metric = "NegLL"

In [117]:
ascending = False if metric=="CI" else True

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{metric}.xlsx")
    excels.extend(glob.glob(glob_search))

for dataset in exclude_datasets:
    for exceli in excels:
        if dataset in exceli:
            print("remove ",dataset)
            excels.remove(exceli)

In [118]:
len(excels)/(len(algorithms)-1)

10.0

In [119]:
df = pd.DataFrame()
N_datasets = len(excels)
percentage_change = []
# print(excels)
for excel_i in excels:

    temp_df = pd.read_excel(excel_i)
    folder_name,dataset_name = excel_i.split("\\")[-3:-1]
    print(folder_name,dataset_name)
    
    temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
    col_name = (dataset_name,re.sub("results_","",folder_name))
    df[col_name] = temp_df[f"Robust {metric}"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)

    col_name = (dataset_name,"baseline")
    df[col_name] = temp_df[f"Non Robust {metric}"]

df.index = temp_df.eps

results_crownibp Aids2
results_crownibp dataDIVAT1
results_crownibp flchain
results_crownibp Framingham
results_crownibp LeukSurv
results_crownibp prostate
results_crownibp retinopathy
results_crownibp stagec
results_crownibp TRACE
results_crownibp zinc
results_fgsm Aids2
results_fgsm dataDIVAT1
results_fgsm flchain
results_fgsm Framingham
results_fgsm LeukSurv
results_fgsm prostate
results_fgsm retinopathy
results_fgsm stagec
results_fgsm TRACE
results_fgsm zinc
results_noise Aids2
results_noise dataDIVAT1
results_noise flchain
results_noise Framingham
results_noise LeukSurv
results_noise prostate
results_noise retinopathy
results_noise stagec
results_noise TRACE
results_noise zinc
results_pgd Aids2
results_pgd dataDIVAT1
results_pgd flchain
results_pgd Framingham
results_pgd LeukSurv
results_pgd prostate
results_pgd retinopathy
results_pgd stagec
results_pgd TRACE
results_pgd zinc


In [120]:
micolumns = pd.MultiIndex.from_tuples(df.columns)
dfmi = (
    pd.DataFrame(
        df.values,
        index=df.index,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)

dfmi.sort_values(by="eps",ascending=False,inplace=True)
dfmi = dfmi.reindex(columns=algorithms, level=1)


In [121]:
dfmi[dfmi==''] = np.NaN
dfmi = dfmi.astype(float)
dfmi

Aids2                                                         \
          baseline          noise          fgsm           pgd    crownibp   
eps                                                                         
1.00  11408.425781  384349.156250  67687.414062  54930.300781  529.279114   
0.90   7220.414062  160613.250000  35049.980469  28619.957031  524.264282   
0.80   4595.645508   66781.250000  17768.054688  14899.712891  522.198975   
0.70   2868.485596   27797.304688   8894.230469   7577.892578  521.521118   
0.60   1798.874268   11356.355469   4351.411621   3794.776367  521.171753   
0.50   1159.978516    4424.974121   2142.486816   1915.282837  520.945190   
0.40    794.657959    1827.487183   1100.876099   1018.304810  520.960388   
0.30    606.031555     853.539429    665.558228    642.262939  521.126709   
0.20    531.618530     564.328491    535.323914    531.216675  521.369202   
0.10    515.993896     519.625244    518.777527    518.681152  521.655640   
0.05    516.784729     519.117188    519.110168    519.044006  521.819092   
0.00    518.514893     520.856506    519.574158    519.484497  521.989746   

         Framingham                                                         \
           baseline         noise          fgsm           pgd     crownibp   
eps                                                                          
1.00  146982.625000  1.265825e+07  1.953815e+06  2.014873e+07  1660.723877   
0.90   70718.859375  3.166705e+06  5.962535e+05  4.359919e+06  1616.299561   
0.80   34176.339844  8.028578e+05  1.836632e+05  9.573399e+05  1582.725342   
0.70   16768.822266  2.061154e+05  5.695138e+04  2.129752e+05  1555.807251   
0.60    8469.119141  5.383214e+04  1.809951e+04  4.876955e+04  1534.328003   
0.50    4569.460938  1.484013e+04  6.330102e+03  1.211749e+04  1517.369385   
0.40    2790.876221  4.919123e+03  2.840475e+03  3.847852e+03  1504.399292   
0.30    2008.951172  2.390979e+03  1.846252e+03  1.973680e+03  1495.161865   
0.20    1675.475342  1.724340e+03  1.579269e+03  1.572052e+03  1489.145874   
0.10    1538.615479  1.541365e+03  1.506851e+03  1.499732e+03  1486.024292   
0.05    1509.514526  1.511206e+03  1.495991e+03  1.493774e+03  1485.464355   
0.00    1497.967529  1.501406e+03  1.492858e+03  1.493241e+03  1485.527222   

      ...        stagec                                              \
      ...      baseline         noise           fgsm            pgd   
eps   ...                                                             
1.00  ...  1.161569e+06  2.070726e+06  179664.765625  153829.218750   
0.90  ...  2.943749e+05  4.931808e+05   54459.613281   47674.171875   
0.80  ...  7.496040e+04  1.143366e+05   16729.857422   14698.337891   
0.70  ...  1.853756e+04  2.733591e+04    5226.172852    4627.388184   
0.60  ...  4.810933e+03  6.494227e+03    1638.094116    1474.163330   
0.50  ...  1.282302e+03  1.608063e+03     530.079346     483.762207   
0.40  ...  3.567604e+02  4.186851e+02     176.607208     168.961960   
0.30  ...  1.138575e+02  1.252089e+02      74.268547      72.265900   
0.20  ...  5.345938e+01  5.504352e+01      45.409916      44.759537   
0.10  ...  4.104847e+01  4.103425e+01      39.572887      39.345528   
0.05  ...  3.975593e+01  3.960278e+01      39.073898      38.911266   
0.00  ...  3.966661e+01  3.950724e+01      39.169571      39.044964   

                           zinc                                            \
         crownibp      baseline         noise          fgsm           pgd   
eps                                                                         
1.00  1032.293945  2.621423e+06  2.929443e+07  2.503320e+06  2.546512e+06   
0.90   578.907715  5.847684e+05  4.721844e+06  4.442481e+05  4.380396e+05   
0.80   330.232391  1.326696e+05  7.761540e+05  9.493627e+04  8.271138e+04   
0.70   194.125397  3.129429e+04  1.320493e+05  2.105466e+04  1.874853e+04   
0.60   120.920013  7.434188e+03  2.336807e+04  4.885672e+03  4.430068e+03   
0.50

In [122]:
# dfmi.T.groupby(axis=0,level=0).rank(axis=0,na_option='bottom',method="average",ascending=ascending)

In [123]:
# for i,sub_df in dfmi.groupby(level=0,axis=1):
#     print(sub_df,sub_df.columns)
#     print(sub_df.rank(axis=1,ascending=ascending))



In [124]:
best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending)
best_per_dataset_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\1243889888.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending)
C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\1243889888.py:1: FutureWarning: DataFrameGroupBy.rank with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset_rank = dfmi.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending)


Aids2                          Framingham                           \
     baseline noise fgsm  pgd crownibp   baseline noise fgsm  pgd crownibp   
eps                                                                          
1.00      2.0   5.0  4.0  3.0      1.0        2.0   4.0  3.0  5.0      1.0   
0.90      2.0   5.0  4.0  3.0      1.0        2.0   4.0  3.0  5.0      1.0   
0.80      2.0   5.0  4.0  3.0      1.0        2.0   4.0  3.0  5.0      1.0   
0.70      2.0   5.0  4.0  3.0      1.0        2.0   4.0  3.0  5.0      1.0   
0.60      2.0   5.0  4.0  3.0      1.0        2.0   5.0  3.0  4.0      1.0   
0.50      2.0   5.0  4.0  3.0      1.0        2.0   5.0  3.0  4.0      1.0   
0.40      2.0   5.0  4.0  3.0      1.0        2.0   5.0  3.0  4.0      1.0   
0.30      2.0   5.0  4.0  3.0      1.0        4.0   5.0  2.0  3.0      1.0   
0.20      3.0   5.0  4.0  2.0      1.0        4.0   5.0  3.0  2.0      1.0   
0.10      1.0   4.0  3.0  2.0      5.0        4.0   5.0  3.0  2.0      1.0   
0.05      1.0   4.0  3.0  2.0      5.0        4.0   5.0  3.0  2.0      1.0   
0.00      1.0   4.0  3.0  2.0      5.0        4.0   5.0  2.0  3.0      1.0   

      ...   stagec                              zinc                           
      ... baseline noise fgsm  pgd crownibp baseline noise fgsm  pgd crownibp  
eps   ...                                                                      
1.00  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  2.0  3.0      1.0  
0.90  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.80  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.70  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.60  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.50  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.40  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.30  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.20  ...      4.0   5.0  3.0  2.0      1.0      4.0   5.0  3.0  2.0      1.0  
0.10  ...      5.0   4.0  3.0  2.0      1.0      5.0   4.0  2.0  1.0      3.0  
0.05  ...      5.0   4.0  3.0  2.0      1.0      4.0   3.0  1.0  2.0      5.0  
0.00  ...      5.0   4.0  3.0  2.0      1.0      1.0   2.0  3.0  4.0      5.0  

[12 rows x 50 columns]

In [125]:
# best_per_dataset_rank.stack(1)

In [126]:
best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="eps",ascending=False)
best_per_dataset_avg_rank

,baseline,noise,fgsm,pgd,crownibp
eps,,,,,
1.00,3.1,4.5,3.4,3.0,1.0
0.90,3.1,4.5,3.5,2.9,1.0
0.80,3.0,4.6,3.5,2.9,1.0
0.70,3.1,4.5,3.5,2.9,1.0
0.60,3.0,4.7,3.5,2.8,1.0
0.50,3.0,4.7,3.5,2.8,1.0
0.40,3.1,4.7,3.4,2.8,1.0
0.30,3.3,4.9,3.2,2.6,1.0
0.20,3.4,4.9,3.4,2.3,1.0


In [112]:
if not ascending:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmax(1)
else:
    best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
best_per_dataset.applymap(lambda x: x[1])

C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\1722459016.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\1722459016.py:4: FutureWarning: DataFrameGroupBy.idxmin with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset = dfmi.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\1722459016.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1])


,Aids2,Framingham,LeukSurv,TRACE,dataDIVAT1,flchain,prostate,retinopathy,stagec,zinc
eps,,,,,,,,,,
1.00,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.90,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.80,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.70,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.60,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.50,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.40,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.30,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp
0.20,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp,crownibp


In [113]:
if metric != "NegLL":
     # aggregate all the CI files
    os.listdir(results_folder)
    excels = []
    for folder in os.listdir(results_folder):
        glob_search = os.path.join(results_folder,folder,f"{metric}_all.xlsx")
        excels.extend(glob.glob(glob_search))
        
    df = pd.DataFrame()
    N_datasets = len(excels)
    percentage_change = []
    for excel_i in excels:
    
        temp_df = pd.read_excel(excel_i)
        folder_name = excel_i.split("\\")[-2]
        temp_df.columns = ["eps"] + temp_df.columns[1:].to_list()
        col_name = re.sub("results_","",folder_name)
        df[col_name] = temp_df["%"] #temp_df["Non Robust CI"].round(3).astype(str) + " / " + temp_df["Robust CI"].round(3).astype(str)
    
    df.index = temp_df.eps
    df = df.reindex(columns=algorithms[1:], level=1)

In [114]:
df

,noise,fgsm,pgd,crownibp
eps,,,,
1.00,1.085146,0.903677,0.879083,-33.617907
0.90,1.744084,1.393828,1.358751,-41.578321
0.80,2.852840,2.213905,2.191946,-46.639185
0.70,4.790237,3.688892,3.704305,-50.721765
0.60,8.234220,5.804617,5.955046,-53.660744
0.50,13.335702,6.949360,7.423757,-54.400784
0.40,17.202435,2.517066,-1.096551,-51.093464
0.30,13.931047,-10.355017,-12.708603,-42.301278
0.20,5.036681,-10.728867,-14.557384,-27.348132


In [115]:
excel_name = os.path.join(results_folder,f"{metric}_all.xlsx")
with pd.ExcelWriter(excel_name) as writer:  
    dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
    best_per_dataset_rank.to_excel(writer,sheet_name="rank")
    best_per_dataset_avg_rank.to_excel(writer,sheet_name="average_rank")
    best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")
    df.to_excel(writer,sheet_name="%")

C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\229516633.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmi.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
C:\Users\lpott\AppData\Local\Temp\ipykernel_26244\229516633.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")


In [20]:
best_per_dataset_avg_rank.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
best_per_dataset_avg_rank.index.name = r"$\epsilon$"
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))

C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\3581541270.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))


\begin{tabular}{llllll}
\toprule
 & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  \\
\midrule
1.00 & 2.6 & 4.11 & 3.88 & 3.22 & 1.0 \\
0.90 & 2.8 & 4.11 & 3.75 & 3.11 & 1.0 \\
0.80 & 2.8 & 4.33 & 3.62 & 3.0 & 1.0 \\
0.70 & 2.8 & 4.33 & 3.62 & 3.0 & 1.0 \\
0.60 & 2.8 & 4.33 & 3.62 & 3.0 & 1.0 \\
0.50 & 2.9 & 4.33 & 3.62 & 2.89 & 1.0 \\
0.40 & 3.1 & 4.67 & 3.25 & 2.67 & 1.0 \\
0.30 & 3.2 & 4.67 & 3.25 & 2.56 & 1.0 \\
0.20 & 3.4 & 4.67 & 3.38 & 2.22 & 1.0 \\
0.10 & 3.0 & 4.44 & 2.88 & 2.33 & 1.9 \\
0.05 & 2.3 & 4.33 & 2.88 & 2.67 & 2.4 \\
0.00 & 2.8 & 3.78 & 3.12 & 2.56 & 2.3 \\
\bottomrule
\end{tabular}



In [21]:
if metric == "NegLL":
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))
else:
    dfmi.index = ["{:.2f}".format(float(x)) for x in np.round(dfmi.index.tolist(),2)]
    dfmi.index.name = r"$\epsilon$"
    print(dfmi.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllllllllll}
\toprule
 & \multicolumn{5}{c}{Aids2} & \multicolumn{5}{c}{Framingham} & \multicolumn{5}{c}{LeukSurv} & \multicolumn{5}{c}{TRACE} & \multicolumn{4}{c}{dataDIVAT1} & \multicolumn{2}{c}{flchain} & \multicolumn{5}{c}{prostate} & \multicolumn{5}{c}{retinopathy} & \multicolumn{5}{c}{stagec} & \multicolumn{5}{c}{zinc} \\
 & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & pgd & crownibp & baseline & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp & baseline & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
1.00 & 0.24 & 0.254 & 0.252 & 0.252 & 0.126 & 0.81 

C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\718377934.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))


In [22]:
dataset_names = np.array(list(map(np.array,dfmi.columns)))[:,0]

In [23]:
dataset_names

array(['Aids2', 'Aids2', 'Aids2', 'Aids2', 'Aids2', 'Framingham',
       'Framingham', 'Framingham', 'Framingham', 'Framingham', 'LeukSurv',
       'LeukSurv', 'LeukSurv', 'LeukSurv', 'LeukSurv', 'TRACE', 'TRACE',
       'TRACE', 'TRACE', 'TRACE', 'dataDIVAT1', 'dataDIVAT1',
       'dataDIVAT1', 'dataDIVAT1', 'flchain', 'flchain', 'prostate',
       'prostate', 'prostate', 'prostate', 'prostate', 'retinopathy',
       'retinopathy', 'retinopathy', 'retinopathy', 'retinopathy',
       'stagec', 'stagec', 'stagec', 'stagec', 'stagec', 'zinc', 'zinc',
       'zinc', 'zinc', 'zinc'], dtype='<U11')

## LONG TABLE

In [24]:
dfmi.T

$\epsilon$                1.00      0.90      0.80      0.70      0.60  \
Aids2       baseline  0.239975  0.233316  0.224340  0.212253  0.196552   
            noise     0.254310  0.252837  0.249959  0.244803  0.235888   
            fgsm      0.252133  0.249117  0.244104  0.235987  0.222965   
            pgd       0.251618  0.248292  0.242862  0.234209  0.220515   
            crownibp  0.125547  0.124267  0.123740  0.123527  0.123394   
Framingham  baseline  0.809922  0.799625  0.777993  0.733339  0.649765   
            noise     0.817904  0.817710  0.816192  0.808553  0.777532   
            fgsm      0.817700  0.816311  0.810436  0.790514  0.728456   
            pgd       0.817916  0.817872  0.817077  0.811265  0.783993   
            crownibp  0.164692  0.151506  0.141450  0.133418  0.127066   
LeukSurv    baseline  0.162709  0.162706  0.162686  0.162570  0.162104   
            noise     0.162709  0.162709  0.162709  0.162709  0.162709   
            fgsm      0.162709  0.162709  0.162709  0.162709  0.162709   
            pgd       0.162709  0.162709  0.162709  0.162709  0.162709   
            crownibp  0.154396  0.149615  0.143566  0.136124  0.127913   
TRACE       baseline  0.634074  0.633944  0.632954  0.625969  0.603408   
            noise     0.634046  0.633769  0.632039  0.622390  0.596380   
            fgsm      0.634090  0.634069  0.633700  0.630725  0.612974   
            pgd       0.634045  0.633673  0.631631  0.620806  0.580239   
            crownibp  0.282150  0.254065  0.231293  0.212871  0.197944   
dataDIVAT1  baseline  0.721355  0.702389  0.670120  0.616419  0.530956   
            noise     0.748570  0.748562  0.748460  0.747749  0.743401   
            pgd       0.748570  0.748570  0.748564  0.748376  0.745912   
            crownibp  0.265944  0.240573  0.223926  0.211592  0.201632   
flchain     baseline  0.830907  0.830907  0.830905  0.830881  0.830512   
            crownibp  0.522363  0.160537  0.075962  0.062986  0.056826   
prostate    baseline  0.529750  0.529349  0.528313  0.525695  0.519496   
            noise     0.530436  0.530436  0.530436  0.530436  0.530436   
            fgsm      0.530436  0.530436  0.530436  0.530424  0.530308   
            pgd       0.530436  0.530436  0.530435  0.530383  0.530162   
            crownibp  0.508492  0.482116  0.441147  0.393497  0.347914   
retinopathy baseline  0.662256  0.656917  0.644868  0.621401  0.582251   
            noise     0.662120  0.656896  0.645055  0.621744  0.582902   
            fgsm      0.656265  0.645536  0.625836  0.596757  0.555833   
            pgd       0.655980  0.644885  0.625174  0.596424  0.554077   
            crownibp  0.526288  0.484388  0.436263  0.384595  0.334449   
stagec      baseline  0.504698  0.504697  0.504666  0.504374  0.501682   
            noise     0.504698  0.504698  0.504692  0.504492  0.501688   
            fgsm      0.504698  0.504695  0.504599  0.503102  0.494473   
            pgd       0.504698  0.504688  0.504509  0.502561  0.492727   
            crownibp  0.490990  0.476400  0.449963  0.407757  0.354486   
zinc        baseline  0.907393  0.906751  0.904731  0.899034  0.881890   
            noise     0.907530  0.907316  0.906561  0.903614  0.892668   
            fgsm      0.907482  0.906870  0.904008  0.892158  0.839130   
            pgd       0.907411  0.906589  0.903191  0.889709  0.831829   
            crownibp  0.719628  0.609333  0.475971  0.352035  0.259768   

$\epsilon$                0.50      0.40      0.30      0.20      0.10  \
Aids2       baseline  0.177556  0.156540  0.136834  0.123815  0.120185   
            noise     0.220413  0.195600  0.160976  0.130051  0.120586   
            fgsm      0.203173  0.175690  0.145125  0.124610  0.120604   
            pgd       0.199893  0.172130  0.142533  0.123809  0.120673   
            crownibp  0.123283  0.123242  0.123270  0.123327  0.123415   
Framingham  baseline  0.520130  0.369269  0.243020  0.165323  0.127692   
   

In [25]:
if metric == "NegLL":
    print(dfmi.T.applymap(lambda x: str(np.round(x,3))).applymap(lambda x: "{:.2e}".format(float(x))).to_latex(index=True,multicolumn_format="c"))
else:
    print(dfmi.T.applymap(lambda x: str(np.round(x,3))).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{llllllllllllll}
\toprule
 & $\epsilon$ & 1.00 & 0.90 & 0.80 & 0.70 & 0.60 & 0.50 & 0.40 & 0.30 & 0.20 & 0.10 & 0.05 & 0.00 \\
\midrule
\multirow[t]{5}{*}{Aids2} & baseline & 0.24 & 0.233 & 0.224 & 0.212 & 0.197 & 0.178 & 0.157 & 0.137 & 0.124 & 0.12 & 0.121 & 0.122 \\
 & noise & 0.254 & 0.253 & 0.25 & 0.245 & 0.236 & 0.22 & 0.196 & 0.161 & 0.13 & 0.121 & 0.121 & 0.122 \\
 & fgsm & 0.252 & 0.249 & 0.244 & 0.236 & 0.223 & 0.203 & 0.176 & 0.145 & 0.125 & 0.121 & 0.121 & 0.121 \\
 & pgd & 0.252 & 0.248 & 0.243 & 0.234 & 0.221 & 0.2 & 0.172 & 0.143 & 0.124 & 0.121 & 0.121 & 0.121 \\
 & crownibp & 0.126 & 0.124 & 0.124 & 0.124 & 0.123 & 0.123 & 0.123 & 0.123 & 0.123 & 0.123 & 0.123 & 0.124 \\
\cline{1-14}
\multirow[t]{5}{*}{Framingham} & baseline & 0.81 & 0.8 & 0.778 & 0.733 & 0.65 & 0.52 & 0.369 & 0.243 & 0.165 & 0.128 & 0.119 & 0.115 \\
 & noise & 0.818 & 0.818 & 0.816 & 0.809 & 0.778 & 0.681 & 0.495 & 0.294 & 0.174 & 0.128 & 0.119 & 0.116 \\
 & fgsm & 0.818 & 0.816 & 0.81 

C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\2645064119.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.T.applymap(lambda x: str(np.round(x,3))).to_latex(index=True,multicolumn_format="c"))


In [26]:
# print(dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().applymap(lambda x: str(np.round(x,3))).reindex(algorithms).to_latex(index=True,multicolumn_format="c"))

if metric == "NegLL":
    print(dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().applymap(lambda x: "{:.2e}".format(float(x))).reindex(algorithms).to_latex(index=True,multicolumn_format="c"))
else:
    print(dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().applymap(lambda x: str(np.round(x,3))).reindex(algorithms).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllllll}
\toprule
$\epsilon$ & 1.00 & 0.90 & 0.80 & 0.70 & 0.60 & 0.50 & 0.40 & 0.30 & 0.20 & 0.10 & 0.05 & 0.00 \\
\midrule
baseline & 0.6 & 0.596 & 0.588 & 0.573 & 0.546 & 0.499 & 0.426 & 0.335 & 0.23 & 0.156 & 0.147 & 0.143 \\
noise & 0.58 & 0.579 & 0.577 & 0.572 & 0.558 & 0.522 & 0.437 & 0.318 & 0.227 & 0.18 & 0.169 & 0.165 \\
fgsm & 0.558 & 0.556 & 0.552 & 0.543 & 0.518 & 0.455 & 0.352 & 0.26 & 0.196 & 0.161 & 0.154 & 0.153 \\
pgd & 0.579 & 0.578 & 0.574 & 0.566 & 0.545 & 0.486 & 0.377 & 0.26 & 0.189 & 0.161 & 0.156 & 0.156 \\
crownibp & 0.376 & 0.313 & 0.274 & 0.242 & 0.213 & 0.189 & 0.17 & 0.156 & 0.146 & 0.141 & 0.14 & 0.14 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\3680683538.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().applymap(lambda x: str(np.round(x,3))).reindex(algorithms).to_latex(index=True,multicolumn_format="c"))


In [27]:
dfmi.T.swaplevel(i=1,j=0,axis=0).groupby(level=0).mean().reindex(algorithms)

$\epsilon$,1.00,0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10,0.05,0.00
baseline,0.600304,0.596060,0.588158,0.573194,0.545862,0.498992,0.425574,0.334527,0.229933,0.155875,0.146720,0.143204
noise,0.580258,0.579437,0.577345,0.571832,0.558178,0.521723,0.437051,0.317809,0.227201,0.180264,0.169183,0.164701
fgsm,0.558189,0.556218,0.551979,0.542797,0.518356,0.455011,0.352199,0.260061,0.195910,0.160744,0.154104,0.152991
pgd,0.579265,0.577524,0.574017,0.566271,0.544685,0.486033,0.376531,0.259695,0.189481,0.160651,0.156466,0.156166
crownibp,0.376049,0.313280,0.274328,0.241840,0.213139,0.189154,0.170441,0.156180,0.146476,0.141207,0.140154,0.139942


In [28]:
dfmi.T.groupby(level=0).mean()

$\epsilon$,1.00,0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10,0.05,0.00
Aids2,0.224716,0.221566,0.217001,0.210156,0.199863,0.184863,0.164640,0.141748,0.125122,0.121093,0.121318,0.121737
Framingham,0.685627,0.680605,0.672630,0.655418,0.613362,0.517226,0.362517,0.219292,0.146352,0.121133,0.116675,0.114967
LeukSurv,0.161046,0.160090,0.158876,0.157364,0.155629,0.153670,0.151779,0.147903,0.137139,0.118506,0.111989,0.109154
TRACE,0.563681,0.557904,0.552323,0.542552,0.518189,0.463336,0.371344,0.271533,0.206014,0.175556,0.168919,0.166203
dataDIVAT1,0.621110,0.610023,0.597768,0.581034,0.555475,0.513675,0.425768,0.283773,0.203465,0.184870,0.183387,0.183731
flchain,0.676635,0.495722,0.453434,0.446934,0.443669,0.442012,0.440762,0.434640,0.268102,0.072970,0.059752,0.050482
prostate,0.525910,0.520554,0.512153,0.502087,0.491663,0.480940,0.467985,0.441727,0.369972,0.282136,0.255089,0.242019
retinopathy,0.632582,0.617724,0.595439,0.564184,0.521903,0.463353,0.382761,0.295491,0.232457,0.202179,0.197289,0.197395
stagec,0.501957,0.499036,0.493686,0.484457,0.469011,0.435492,0.372433,0.278208,0.187653,0.165954,0.171663,0.180793
zinc,0.869889,0.847372,0.818892,0.787310,0.741057,0.631646,0.429618,0.240246,0.141579,0.101361,0.092929,0.089193


In [29]:
# pd.set_option('display.max_columns', None)
# print(dfmi)

In [30]:
print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))

\begin{tabular}{llllllllll}
\toprule
Aids2 & Framingham & LeukSurv & TRACE & dataDIVAT1 & flchain & prostate & retinopathy & stagec & zinc \\
\midrule
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crownibp \\
crownibp & crownibp & crownibp & crownibp & crownibp & crownibp & crowni

C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\3574660769.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset.applymap(lambda x: x[1]).to_latex(index=False))


In [31]:
df.index = ["{:.2f}".format(float(x)) for x in np.round(df.index.tolist(),2)]
df.index.name = r"$\epsilon$"
print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))

\begin{tabular}{lllll}
\toprule
 & noise & fgsm & pgd & crownibp \\
$\epsilon$ &  &  &  &  \\
\midrule
1.00 & 1.206 & 0.658 & 0.977 & -33.618 \\
0.90 & 1.938 & 0.921 & 1.51 & -41.578 \\
0.80 & 3.17 & 1.309 & 2.437 & -46.639 \\
0.70 & 5.322 & 1.968 & 4.124 & -50.722 \\
0.60 & 9.149 & 2.346 & 6.625 & -53.661 \\
0.50 & 14.824 & 0.26 & 8.697 & -54.401 \\
0.40 & 19.112 & -4.722 & 5.902 & -51.093 \\
0.30 & 15.633 & -7.376 & -4.56 & -42.301 \\
0.20 & 10.991 & -4.61 & -6.865 & -27.348 \\
0.10 & 8.78 & -0.748 & -1.875 & -10.317 \\
0.05 & 7.193 & 0.475 & 0.253 & -5.206 \\
0.00 & 5.964 & 1.415 & 1.479 & -1.684 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_22952\3740848601.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df.applymap(lambda x: str(np.round(x,3))).to_latex(index=True))


## PDF PICTURES

In [16]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import pandas as pd
import re

In [17]:
attack = "crownibp"

In [18]:
results_folder = rf"results\{attack}"

In [19]:
algorithms = ["baseline","noise","fgsm","pgd","crownibp"]

In [20]:
# CI , IBS , NegLL
img_name = "population_curves_attacked_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

for dataset in exclude_datasets:
    for exceli in excels:
        if dataset in exceli:
            print("remove ",dataset)
            excels.remove(exceli)

In [21]:
metadata = 'eps'

In [22]:
n_rows = int(len(excels) / (len(algorithms)-1))
n_cols = int(len(algorithms) - 1)

In [23]:
excels

['results\\crownibp\\results_crownibp\\Aids2\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\dataDIVAT1\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\flchain\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\Framingham\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\LeukSurv\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\prostate\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\retinopathy\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\stagec\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\TRACE\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_crownibp\\zinc\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_fgsm\\Aids2\\population_curves_attacked_test.xlsx',
 'results\\crownibp\\results_fgsm\\dataDIVAT1

In [25]:
len(excels)/(len(algorithms)-1)

10.0

In [39]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

ValueError: cannot reshape array of size 35 into shape (4)

In [ ]:
df_population = pd.read_excel(excels[0])

In [ ]:
df_population

In [ ]:
save_folder = results_folder = os.path.join(fr"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results\{attack}","perturb_curves.pdf")

In [ ]:
print(save_folder)

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves.pdf")

fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,64),sharey=True)

SMALL_SIZE = 80
plt.rc('xtick', labelsize=SMALL_SIZE//2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE//2)  
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)   # fontsize of the figure title


for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]

        # print(base_models)
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"{dataset}\n S(t)" ,fontsize=SMALL_SIZE//1.5)
            axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

        axes[i][j+1].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=SMALL_SIZE//1.5)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=5,fontsize=30,bbox_to_anchor=(.5,1.06),prop={'size':SMALL_SIZE//1.7}) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout(pad=0)
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
files[[-7,-2,-1],:]

In [ ]:
save_folder = results_folder = os.path.join(r"C:\Users\lpott\OneDrive\Documents\Northeastern University\Classes\EECE 7398 Verifiable Machine Learning\EECE7398_Final_Project\results",attack,"perturb_curves_subset.pdf")
file_subset = files[[-7,-2,-1],:]

fig,axes = plt.subplots(3,n_cols+1,figsize=(30,20),sharey=True)


SMALL_SIZE = 80
plt.rc('xtick', labelsize=SMALL_SIZE//2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE//2)  
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)   # fontsize of the figure title

for i in range(3):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        if dataset not in ["TRACE","stagec","zinc"]:
            continue
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        df_population= df_population.iloc[:,3:]

        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in robust_df.columns]
        baseline_df.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in baseline_df.columns]
        
        base1 = axes[i][j+1].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j+1].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        base3 = axes[i][j+1].plot(t,base_models.iloc[:,2],linewidth=3,c="k")

  
        robust1 = axes[i][j+1].plot(t,robust_df,'--',linewidth=3)
  
        if j == 0:
            base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            base3 = axes[i][j].plot(t,base_models.iloc[:,2],linewidth=3,c="k")
            baseline1 = axes[i][j].plot(t,baseline_df,'--',linewidth=3)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=SMALL_SIZE//1.5)
            axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

        axes[i][j+1].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=SMALL_SIZE//1.5)


labels = base_models.columns.tolist() + robust_df.columns.tolist()
fig.legend([base1, base2,base3,robust1], labels=labels, 
           loc="upper center",ncols=6,fontsize=30,bbox_to_anchor=(.5,1.11),prop={'size':SMALL_SIZE}) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout(pad=0)
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

### DIST PLOTS

In [ ]:
# CI , IBS , NegLL
results_folder = fr"results\{attack}"
img_name = "curve_distributions_test"

 # aggregate all the CI files
os.listdir(results_folder)
excels = []
for folder in os.listdir(results_folder):
    glob_search = os.path.join(results_folder,folder,"*",f"{img_name}.xlsx")
    excels.extend(glob.glob(glob_search))

for dataset in exclude_datasets:
    for exceli in excels:
        if dataset in exceli:
            print("remove ",dataset)
            excels.remove(exceli)

In [ ]:
excels = np.sort(excels)
files = pd.DataFrame(excels.reshape(-1,len(algorithms)-1,order="F"),columns=["crownibp","fgsm","noise","pgd"]).reindex(["noise","fgsm","pgd","crownibp"],axis=1).values

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves.pdf")

In [ ]:
import seaborn as sns

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols+1,figsize=(30,64),sharey=True)

SMALL_SIZE = 80
plt.rc('xtick', labelsize=SMALL_SIZE//2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE//2)  
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)   # fontsize of the figure title


for i in range(n_rows):
    for j in range(n_cols):
        print(i,j)
        fileij = files[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        axes[i][j+1].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"{dataset}\n S(t)" ,fontsize=SMALL_SIZE//1.5)
            axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=SMALL_SIZE//1.5)

labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.05),prop={'size':SMALL_SIZE}) 

plt.tight_layout(pad=0)
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

In [ ]:
save_folder = results_folder = os.path.join(r"results",attack,"dist_curves_subset.pdf")
file_subset = files[[-7,-2,-1],:]

fig,axes = plt.subplots(3,n_cols+1,figsize=(30,20),sharey=True)

SMALL_SIZE = 20
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  

for i in range(3):
    for j in range(n_cols):
        print(i,j)
        fileij = file_subset[i][j]
        algo,dataset = fileij.split("\\")[-3:-1]
        algo = re.sub("results_","",algo)

        df_population = pd.read_excel(fileij)

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:3]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        
        robust_idx = ["robust" in col for col in df_population.columns]
        baseline_idx = ["baseline" in col for col in df_population.columns]
        robust_df = df_population.iloc[:,robust_idx]
        baseline_df = df_population.iloc[:,baseline_idx]
        
        robust_df.columns = [col.split("_")[1] for col in robust_df.columns]
        baseline_df.columns =[col.split("_")[1] for col in baseline_df.columns]
        
        mu = sns.lineplot(x=t, y=robust_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j+1],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=robust_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=robust_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j+1],c='r',legend=False)
        axes[i][j+1].set_xlabel("t",fontsize=20)

        line = q05.get_lines()
        axes[i][j+1].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        if j == 0:
            mu = sns.lineplot(x=t, y=baseline_df.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
            q95 = sns.lineplot(x=t, y=baseline_df.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
            q05 = sns.lineplot(x=t, y=baseline_df.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

            axes[i][j].set_ylabel(f"S(t) {dataset}" ,fontsize=30)
            axes[i][j].set_xlabel("t",fontsize=20)
            line = q05.get_lines()
            axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
            axes[i][j].set_xlabel("t",fontsize=20)

for ax, col in zip(axes[0], algorithms):
    ax.set_title(col,fontsize=30)


labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.05)) 

plt.tight_layout()
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()